In [21]:
#Import dependencies
from geopy.geocoders import GoogleV3
import pandas as pd, requests, time
import warnings
import geopandas as gpd
import sqlite3
from sqlalchemy import create_engine

# import api key
from config import places_key

# 1. Setup
### 1.1 Import Data

In [5]:

df = pd.read_csv('Resources/Meteorite_Landings.csv')
df.head(5)

,name,id,nametype,recclass,mass (g),fall,year,latitude,longitude,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)"


In [6]:
# Examine data type of each column
df.dtypes

name            object
id               int64
nametype        object
recclass        object
mass (g)       float64
fall            object
year           float64
latitude       float64
longitude      float64
GeoLocation     object
dtype: object

In [7]:
# Check what types of meteorites are in the 'fall' column
df.fall.unique()

array(['Fell', 'Found'], dtype=object)

# 2. Data Cleaning
### 2.1 Remove Unwanted Columns and Rows

In [8]:
# The column "fall" indicates whether the meteorite was observed falling or was found at a location
df.fall.unique()

array(['Fell', 'Found'], dtype=object)

In [9]:
# For our purposes, we want the ones observed falling at locations
df2 = df[df.fall == 'Fell']

# Check to make sure 'fell' meteorites are the only ones left
df2.fall.unique()

array(['Fell'], dtype=object)

In [10]:
# Since we only have fell objects now, we don't need the "fall" column. 
# We also don't need the "nametype" column, so we're going to drop those
df2 = (df2.drop(['nametype', 'fall'], axis=1).dropna())
df2.head()

,name,id,recclass,mass (g),year,latitude,longitude,GeoLocation
0,Aachen,1,L5,21.0,1880.0,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,H6,720.0,1951.0,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,EH4,107000.0,1952.0,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Acapulcoite,1914.0,1976.0,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,L6,780.0,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)"


### 2.2 Filter Out Meterorites that Fell Outside the USA

In [11]:
# We only want meteorites that fell within the USA, 
# so we need to figure out what countries the meteorites fell over using GeoLocation as the key

# Create a key in the main dataframe by using a format for GeoLocation that the Google Places API likes
df2.GeoLocation = (
    df2
    .GeoLocation
    .str
    .strip()
    .replace(
            {
            '\(':'',
            '\)':'',
            ' ':''
            },
        regex=True)
)
df2.head()

,name,id,recclass,mass (g),year,latitude,longitude,GeoLocation
0,Aachen,1,L5,21.0,1880.0,50.77500,6.08333,"50.775,6.08333"
1,Aarhus,2,H6,720.0,1951.0,56.18333,10.23333,"56.18333,10.23333"
2,Abee,6,EH4,107000.0,1952.0,54.21667,-113.00000,"54.21667,-113.0"
3,Acapulco,10,Acapulcoite,1914.0,1976.0,16.88333,-99.90000,"16.88333,-99.9"
4,Achiras,370,L6,780.0,1902.0,-33.16667,-64.95000,"-33.16667,-64.95"


In [12]:
# To save time during our api request, narrow down the key to only unique locations
geo_key = pd.DataFrame(df2['GeoLocation'].unique(),columns=['GeoLocation'])
geo_key

,GeoLocation
0,"50.775,6.08333"
1,"56.18333,10.23333"
2,"54.21667,-113.0"
3,"16.88333,-99.9"
4,"-33.16667,-64.95"
...,...
1059,"39.66667,122.98333"
1060,"51.83333,26.83333"
1061,"-15.18333,35.28333"
1062,"46.93333,21.5"


In [13]:
# Set the pause duration between api requests
pause = 0.1

In [14]:
# Set up a function that accepts a latitude,longitude string, sends it to the Google API, and returns the location
def geocode(latlong):
    time.sleep(pause) #pause for some duration before each request, to not hammer their server
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&sensor=false&key='+places_key #api url with placeholders
    request = url.format(latlong) #fill in the placeholder with a variable
    response = requests.get(request) #send the request to the server and get the response
    data = response.json() #convert the response json string into a dict
    
    if len(data['results']) > 0: #if google was able to geolocate our address, extract formatted address from result
        return data['results'][0]['formatted_address']

In [15]:
# for each value in the GeoLocation column, geocode it, save results as new df column
geo_key['Address'] = geo_key['GeoLocation'].map(geocode)

In [16]:
#Drop any incomplete sets
geo_key=geo_key.dropna()

# Check how it looks
geo_key.head()

,GeoLocation,Address
0,"50.775,6.08333","Katschhof 2, 52062 Aachen, Germany"
1,"56.18333,10.23333","Tjørnevej 12, 8240 Risskov, Denmark"
2,"54.21667,-113.0","6282+M2 Abee, AB, Canada"
3,"16.88333,-99.9","Vicente Guerrero 18, Parque el Veladero, 39510..."
4,"-33.16667,-64.95","R2MX+8X Achiras, Córdoba, Argentina"


In [17]:
# Filter out all non-USA results
country = 'USA'
geo_key2=geo_key[geo_key['Address'].str.contains(country)]
geo_key2['Country']='USA'
geo_key2.head()

/var/folders/b9/y7yy3gjx1vjcj8dmj6zw15kw0000gn/T/ipykernel_25083/2735779226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_key2['Country']='USA'


,GeoLocation,Address,Country
27,"42.53333,-85.88333","1596 34th St, Allegan, MI 49010, USA",USA
33,"44.61667,-70.75","J782+M2 Andover, ME, USA",USA
43,"38.5,-94.3","33900 S Dickey Rd, Archie, MO 64725, USA",USA
46,"31.805,-97.01","RX4R+22 Cottonwood, TX, USA",USA
51,"34.75,-87.0","Q222+22 Athens, AL, USA",USA


In [18]:
#Merge the key back in with the dataset using an inner join to filter out non-USA meteorites
df3 = df2.merge(geo_key2, on='GeoLocation', how='inner')

#Drop columns not needed from key 
df3 = (df3.drop(['GeoLocation', 'Country'], axis=1).dropna())
df3.head()

,name,id,recclass,mass (g),year,latitude,longitude,Address
0,Allegan,2276,H5,32000.0,1899.0,42.53333,-85.88333,"1596 34th St, Allegan, MI 49010, USA"
1,Andover,2295,L6,3200.0,1898.0,44.61667,-70.75000,"J782+M2 Andover, ME, USA"
2,Archie,2329,H6,5070.0,1932.0,38.50000,-94.30000,"33900 S Dickey Rd, Archie, MO 64725, USA"
3,Ash Creek,48954,L6,9500.0,2009.0,31.80500,-97.01000,"RX4R+22 Cottonwood, TX, USA"
4,Athens,4885,LL6,265.0,1933.0,34.75000,-87.00000,"Q222+22 Athens, AL, USA"


In [23]:
#Make the db in memory
connection_string = "Resources/Meteorite_Landings.sqlite"
engine = create_engine(f"sqlite:///{connection_string}")

#write the tables
df2.to_sql('df2', con=engine, if_exists='replace', index=False)
geo_key2.to_sql('geo_key2', con=engine, if_exists='replace', index=False)

qry = '''
    SELECT  
        df2.name,
        df2.id,
        df2.recclass,
        df2.year,
        df2.latitude,
        df2.longitude,
        geo_key2.Address
    FROM df2
        LEFT JOIN geo_key2 ON
        df2.GeoLocation = geo_key2.GeoLocation
    '''
df4 = pd.read_sql_query(qry, con=engine)
df4.head()

,name,id,recclass,year,latitude,longitude,Address
0,Aachen,1,L5,1880.0,50.77500,6.08333,None
1,Aarhus,2,H6,1951.0,56.18333,10.23333,None
2,Abee,6,EH4,1952.0,54.21667,-113.00000,None
3,Acapulco,10,Acapulcoite,1976.0,16.88333,-99.90000,None
4,Achiras,370,L6,1902.0,-33.16667,-64.95000,None


In [ ]:
#Save as JSON
df4.to_json('Resources/Meteorite_Landings.json')

In [ ]:
# Set the Coordinate Reference System (CRS) to the WGS84 (epsg:4326) latitude-longitude projection 
# fyi: CRS tells Python how coordinates relate to places on the Earth
#      https://spatialreference.org/ref/epsg/4326/
#      https://en.wikipedia.org/wiki/World_Geodetic_System
crs={'init':'epsg:4326'}

# Create a GeoDataFrame using the coordinates
geo_df=gpd.GeoDataFrame(df3,crs=crs,geometry=gpd.points_from_xy(df3['longitude'], df3['latitude']))

# Check dataframe
geo_df

In [ ]:
#Save as a GeoJSON
geo_df.to_file('Project-3-/Resources/Meteorite_Landings.geojson', driver="GeoJSON")  